# Excercise 4: Transfer Learning with Pre-trained CNN

Use a pre-trained convolutional neural network (CNN) as a feature extractor and finetune a classifier on a subset of the CIFAR-10 dataset. Set random seeds to 42. Follow
the configuration below:

- Load CIFAR-10 and normalize pixel values to [0,1]
- Use only the first 2000 training samples and first 500 test samples
- Load MobileNetV2 from tensorflow.keras.applications, with include top=False
and weights=’imagenet’
- Freeze all layers of the pre-trained base
- Add a classifier on top:
  - GlobalAveragePooling2D
  - Dense layer with 128 neurons, ReLU activation
  - Dropout: 0.2
  - Output layer: 10 neurons with softmax
- Optimizer: Adam, learning rate = 0.001
- Loss: sparse categorical crossentropy
- epochs = 5, batch size = 32

Q4.1 Report the test accuracy of the model.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


In [ ]:
# Load and unpack CIFAR-10 dataset
cifar = tf.keras.datasets.cifar10
(X_full, y_full), (X_test_full, y_test_full) = cifar.load_data()

# Use only first 2,000 instances for training/validation split
X_small = X_full[:2000]
y_small = y_full[:2000]

# Split into 1,600 train and 400 validation
X_train, X_valid = X_small[:1600], X_small[1600:]
y_train, y_valid = y_small[:1600], y_small[1600:]

# Use only first 500 test samples
X_test = X_test_full[:500]
y_test = y_test_full[:500]

In [ ]:
# Preprocess the data: normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_valid = X_valid.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
# Resize to MobileNetV2 input size (96x96)
IMG_SIZE = 96
X_train = tf.image.resize(X_train, (IMG_SIZE, IMG_SIZE))
X_valid = tf.image.resize(X_valid, (IMG_SIZE, IMG_SIZE))
X_test = tf.image.resize(X_test, (IMG_SIZE, IMG_SIZE))

In [ ]:
from keras import layers, models

# Load pre-trained CNN
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False  # Freeze all layers

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Add classifier on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_96             │ (None, 3, 3, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 165,258 (645.54 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 25s 374ms/step - accuracy: 0.3358 - loss: 2.0626 - val_accuracy: 0.6480 - val_loss: 1.0277
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 14s 289ms/step - accuracy: 0.6761 - loss: 0.8922 - val_accuracy: 0.6760 - val_loss: 0.9279
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 22s 325ms/step - accuracy: 0.7464 - loss: 0.6937 - val_accuracy: 0.6980 - val_loss: 0.8953
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 344ms/step - accuracy: 0.8222 - loss: 0.5424 - val_accuracy: 0.6800 - val_loss: 0.8766
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 410ms/step - accuracy: 0.8556 - loss: 0.4496 - val_accuracy: 0.7060 - val_loss: 0.8408


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_acc)

Test Accuracy: 0.7059999704360962
